# Importing Modules

In [40]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, ReLU
from keras import layers, Sequential
from tensorflow import matmul, Variable
from tensorflow import math
import warnings
warnings.filterwarnings('ignore')

# Importing Data
- Data is of Rainfall in months of June to September of years 1948 - 2020

In [2]:
import statistics

rain = pd.read_csv("./Data/Rainfall.csv")

lpa = statistics.mean(list(rain['Jun-Sep'][12:41]))
rain_fall_data = list(rain['Jun-Sep'])

rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

- Importing Sea-Level Pressure(SLP), Sea Surface Temperature(SST), and Zonal Wind(UWND) Data.

In [3]:
import glob

slp_path  = "./Data/Anomaly/slp/*"
sst_path  = "./Data/Anomaly/sst/*"
uwnd_path = "./Data/Anomaly/uwnd/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)

sst_months = []
for i in glob.glob(sst_path):
  sst_months.append(i)

uwnd_months = []
for i in glob.glob(uwnd_path):
  uwnd_months.append(i)
  
slp_months.sort()
sst_months.sort()
uwnd_months.sort()

- Start and End Index for data (i.e 1948's index and 2000's index)

In [4]:
s = 0
e = 52

- Extracting data of all the months and stacking one above other. 
- 324 variables and 52 data points for SLP and UWND.
- 192 variables and 52 data points for SST.

In [5]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d))
  return np.array(new_data)

# Stacking Data of All Months
combine_slp, combine_sst, combine_uwnd = [], [], []
for i in range(12):
  month_path_slp, month_path_sst, month_path_uwnd = slp_months[i], sst_months[i], uwnd_months[i] 
  month_data_slp ,month_data_sst, month_data_uwnd = pd.read_csv(month_path_slp, header=[0, 1], index_col=0), pd.read_csv(month_path_sst, header=[0, 1], index_col=0), pd.read_csv(month_path_uwnd, header=[0, 1], index_col=0)
 
  feature_slp, feature_sst, feature_uwnd = get_feature(month_data_slp), get_feature(month_data_sst), get_feature(month_data_uwnd)
  feature_slp, feature_sst, feature_uwnd = feature_slp[s:e], feature_sst[s:e], feature_uwnd[s:e]
  for i in feature_slp:
    combine_slp.append(i)
  for i in feature_sst:
    combine_sst.append(i)
  for i in feature_uwnd:
    combine_uwnd.append(i)

combine = [np.array(combine_slp), np.array(combine_sst), np.array(combine_uwnd)]
month_data_slp.head()

Latitude         -90                                                         \
Longitude          0         20         40         60         80        100   
1949       -3.649323  -3.649323  -3.649323  -3.649323  -3.649323  -3.649323   
1950        1.230677   1.230677   1.230677   1.230677   1.230677   1.230677   
1951        7.090677   7.090677   7.090677   7.090677   7.090677   7.090677   
1952        8.300677   8.300677   8.300677   8.300677   8.300677   8.300677   
1953       10.240677  10.240677  10.240677  10.240677  10.240677  10.240677   

Latitude                                               ...        80  \
Longitude        120        140        160        180  ...       160   
1949       -3.649323  -3.649323  -3.649323  -3.649323  ...  6.598676   
1950        1.230677   1.230677   1.230677   1.230677  ... -4.251374   
1951        7.090677   7.090677   7.090677   7.090677  ...  0.478676   
1952        8.300677   8.300677   8.300677   8.300677  ...  8.528676   
1953       10.240677  10.240677  10.240677  10.240677  ... -1.301354   

Latitude                                                               \
Longitude       180       200       220       240       260       280   
1949       5.294682  3.563006  1.995687  1.005682  0.951014  1.386347   
1950      -4.835348 -5.186994 -4.854313 -2.824318 -0.829026 -1.153683   
1951       3.114682  5.042976  5.235657  5.285642  5.451014  5.246347   
1952       9.234682  9.053006  7.765687  5.885682  4.100964  2.876347   
1953      -1.155358 -1.876994 -3.864313 -5.174318 -4.628986 -2.843693   

Latitude                                
Longitude      300       320       340  
1949       1.43035  1.857351  3.877972  
1950      -1.92965 -1.202699  0.617962  
1951       5.95035  6.487301  4.937972  
1952       2.64035  3.047301  2.927962  
1953      -2.32968 -3.162649 -2.762038  

[5 rows x 324 columns]

- Concanating the Data as required.

In [6]:
def get_data(List):
    N = len(List)
    if N == 1:
        return combine[List[0]]
    elif N == 2:
        return np.concatenate((combine[List[0]], combine[List[1]]), axis=1)
    else:
        return np.concatenate((combine[List[0]], combine[List[1]], combine[List[2]]), axis=1)


# Models 

## Auto-Encoders (SLP or UWND)

- 324 - 97 - 323

In [7]:
def get_model_324(n_inputs=324):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(97)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 97 - 29 - 97

In [8]:
def get_model_97(n_inputs=97):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(29)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 29 - 9 - 29

In [9]:
def get_model_29(n_inputs=29):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(9)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

#### Final Auto Encoder
- 324 - 97 - 29 - 9 - 29 - 97 - 324

In [10]:
def get_model_final_324(n_inputs=324):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    e = Dense(97)(e)
    e = ReLU()(e)
    e = Dense(29)(e)
    e = ReLU()(e)
    bottleneck = Dense(9)(e)
    e = Dense(29)(bottleneck)
    e = ReLU()(e)
    e = Dense(97)(e)
    e = ReLU()(e)
    output = Dense(n_inputs, activation='linear')(e)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

## Auto-Encoders (SST)

- 192 - 72 - 192

In [11]:
def get_model_192(n_inputs=192):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(72)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 72 - 21 - 72

In [12]:
def get_model_72(n_inputs=72):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(21)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 21 - 6 - 21

In [13]:
def get_model_21(n_inputs=21):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(6)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

#### Final Auto Encoder
- 192 - 72 - 21 - 6 - 21 - 72 - 192

In [14]:
def get_model_final_192(n_inputs=192):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    e = Dense(72)(e)
    e = ReLU()(e)
    e = Dense(21)(e)
    e = ReLU()(e)
    bottleneck = Dense(6)(e)
    e = Dense(21)(bottleneck)
    e = ReLU()(e)
    e = Dense(72)(e)
    e = ReLU()(e)
    output = Dense(n_inputs, activation='linear')(e)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

## Auto-Encoders (SST-SLP or SST-UWND)

- 416 - 169 - 416

In [15]:
def get_model_416(n_inputs=416):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(169)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 169 - 50 - 169

In [16]:
def get_model_169(n_inputs=169):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(50)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 50 - 15 - 50

In [17]:
def get_model_50(n_inputs=50):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(15)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

#### Final Auto Encoder
- 416 - 169 - 50 - 15 - 50 - 169 - 416

In [18]:
def get_model_final_416(n_inputs=416):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    e = Dense(169)(e)
    e = ReLU()(e)
    e = Dense(21)(e)
    e = ReLU()(e)
    bottleneck = Dense(6)(e)
    e = Dense(21)(bottleneck)
    e = ReLU()(e)
    e = Dense(169)(e)
    e = ReLU()(e)
    output = Dense(n_inputs, activation='linear')(e)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

## Auto-Encoders (SLP-UWND)

- 648 - 194 - 648

In [19]:
def get_model_648(n_inputs=648):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(194)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 194 - 58 - 194

In [20]:
def get_model_194(n_inputs=194):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(58)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

- 58 - 17 - 58

In [21]:
def get_model_58(n_inputs=58):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    bottleneck = Dense(17)(e)
    output = Dense(n_inputs, activation='linear')(bottleneck)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

#### Final Auto Encoder
- 648 - 194 - 58 - 17 - 58 - 194 - 648

In [22]:
def get_model_final_648(n_inputs=648):
    visible = Input(shape=(n_inputs,))
    e = Dense(n_inputs)(visible)
    e = ReLU()(e)
    e = Dense(194)(e)
    e = ReLU()(e)
    e = Dense(58)(e)
    e = ReLU()(e)
    bottleneck = Dense(17)(e)
    e = Dense(58)(bottleneck)
    e = ReLU()(e)
    e = Dense(194)(e)
    e = ReLU()(e)
    output = Dense(n_inputs, activation='linear')(e)
    model = Model(inputs=visible, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

## Get all Models in a 2-D List
|       Outer Most Layer           |        Middle Layer             |      Inner Most Layer         |      Full Model                              |
|----------------------------------|---------------------------------|-------------------------------|----------------------------------------------|
| [ Model SLP  - 324-97-324        |  Model SLP  - 97-29-97          |  Model SLP  - 29-9-29         |  Model SLP  - 324-97-29-9-29-97-324  ]       |
| [ Model UWND - 324-97-324        |   Model UWND - 97-29-97         |   Model UWND - 29-9-29        |   Model UWND - 324-97-29-9-29-97-324 ]       |
| [ Model SST  - 192-72-192        |   Model SST  - 72-21-72         |   Model SST  - 21-6-21        |   Model SST  - 192-72-21-6-21-72-192 ]       |
| [ Model SLP-SST  - 416-169-416   |   Model SLP-SST  - 169-50-169   |   Model SLP-SST  - 50-15-50   |   Model SLP-SST  - 416-169-50-15-50-169-416] |
| [ Model UWND-SST - 416-169-416   |   Model UWND-SST - 169-50-169   |   Model UWND-SST - 50-15-50   |   Model UWND-SST - 416-169-50-15-50-169-416] |
| [ Model SLP-UWND - 648-194-648   |   Model SLP-UWND - 194-58-194   |   Model SLP-UWND - 58-17-58   |   Model SLP-UWND - 416-194-58-17-58-194-416] |

In [23]:
def get_all_model():
    models_slp  = [get_model_324(), get_model_97(), get_model_29(), get_model_final_324()]
    models_uwnd = [get_model_324(), get_model_97(), get_model_29(), get_model_final_324()]
    models_sst  = [get_model_192(), get_model_72(), get_model_21(), get_model_final_192()]
    models_slp_sst  = [get_model_416(), get_model_169(), get_model_50(), get_model_final_416()]
    models_uwnd_sst = [get_model_416(), get_model_169(), get_model_50(), get_model_final_416()]
    models_slp_uwnd = [get_model_648(), get_model_194(), get_model_58(), get_model_final_648()]

    models = [models_slp, models_uwnd, models_sst, models_slp_sst, models_uwnd_sst, models_slp_uwnd]
    return models

In [24]:
models = get_all_model()

In [25]:
for i in models[0]:
    i.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 324)]             0         
                                                                 
 dense (Dense)               (None, 324)               105300    
                                                                 
 re_lu (ReLU)                (None, 324)               0         
                                                                 
 dense_1 (Dense)             (None, 97)                31525     
                                                                 
 dense_2 (Dense)             (None, 324)               31752     
                                                                 
Total params: 168,577
Trainable params: 168,577
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
______________________________________

# Training

In [26]:
models = get_all_model()
data = np.concatenate((get_data([0]), get_data([2])), axis=0)
# data = get_data([0])
data.shape

(1248, 324)

In [31]:
def get_next_inputs(models, index, inputs):
    model_temp = Model(inputs=models[index].input, outputs=models[index].layers[3].output)
    return model_temp(inputs)

def train_all_models(models, inputs):
    x = inputs
    if type(models) == type([]):
        for i, model in enumerate(models[:-1]):
            model.fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))
            x = get_next_inputs(models, i, x)

        models[-1].layers[2] = models[0].layers[3]
        models[-1].layers[11]= models[0].layers[4]
        models[-1].layers[4] = models[1].layers[3]
        models[-1].layers[9] = models[1].layers[4]
        models[-1].layers[6] = models[2].layers[3]
        models[-1].layers[7] = models[2].layers[4]
        models[-1].compile(optimizer='adam', loss='mse')

        model[-1].fit(inputs, inputs, epochs=800, batch_size=16, verbose=2, validation_data=(inputs,inputs))

def train_perticular_model(models, index, inputs):
    if not index == 0 and not index == -1:
        x = get_next_inputs(models, index-1, inputs)
    else:
        x = inputs
    models[index].fit(x, x, epochs=800, batch_size=16, verbose=2, validation_data=(x,x))

In [32]:
train_perticular_model(models[0], -1, data)

Epoch 1/800
78/78 - 0s - loss: 17.6789 - val_loss: 15.3070 - 467ms/epoch - 6ms/step
Epoch 2/800
78/78 - 0s - loss: 14.5371 - val_loss: 13.8178 - 118ms/epoch - 2ms/step
Epoch 3/800
78/78 - 0s - loss: 13.6829 - val_loss: 13.2415 - 117ms/epoch - 1ms/step
Epoch 4/800
78/78 - 0s - loss: 13.2030 - val_loss: 12.8798 - 117ms/epoch - 1ms/step
Epoch 5/800
78/78 - 0s - loss: 12.8430 - val_loss: 12.5198 - 121ms/epoch - 2ms/step
Epoch 6/800
78/78 - 0s - loss: 12.4618 - val_loss: 12.1043 - 132ms/epoch - 2ms/step
Epoch 7/800
78/78 - 0s - loss: 12.1253 - val_loss: 11.8168 - 125ms/epoch - 2ms/step
Epoch 8/800
78/78 - 0s - loss: 11.8481 - val_loss: 11.5353 - 121ms/epoch - 2ms/step
Epoch 9/800
78/78 - 0s - loss: 11.6131 - val_loss: 11.3501 - 122ms/epoch - 2ms/step
Epoch 10/800
78/78 - 0s - loss: 11.3887 - val_loss: 11.1361 - 119ms/epoch - 2ms/step
Epoch 11/800
78/78 - 0s - loss: 11.2086 - val_loss: 11.0055 - 122ms/epoch - 2ms/step
Epoch 12/800
78/78 - 0s - loss: 11.0989 - val_loss: 10.8884 - 121ms/epoch 

# Post Training Treatment

In [35]:
class Custom_Layer(layers.Layer):
    def __init__(self, weights, bias=False):
        super(Custom_Layer, self).__init__()
        self.w = weights
        if bias:
            self.b = bias

    def call(self, inputs):
        try: return matmul(inputs, self.w) + self.b
        except: return matmul(inputs, self.w)

In [37]:
print([[] if i in [1, 3, 4] else None for i in range(6)])

[None, [], None, [], [], None]


In [52]:
def std_and_discard(indexes):
    final_models = [models[i][-1] if i in indexes else None for i in range(6)]
    std = [[] if i in indexes else None for i in range(6)]
    for i, model in enumerate(final_models):
        if model is not None:
            for layer in model.layers:
                try: std[i].append(np.array(math.reduce_std(layer.weights[0])) * 2)
                except: std[i].append([])

    for index, model in enumerate(final_models):
        if model is not None:
            final_layers = Sequential([Input(shape=(None, 324))])
            for i, layer in enumerate(model.layers):
                try:
                    temp_layer = []
                    for j in layer.weights[0]:
                        temp_weights = []
                        for k in j:
                            if k < std[index][i]: temp_weights.append(0)
                            else:          temp_weights.append(1)  
                        temp_layer.append(temp_weights)
                    weights = Variable(temp_layer, dtype='float32')
                    try: final_layers.add(Custom_Layer(weights, bias=layer.weights[1]))
                    except: final_layers.add(Custom_Layer(weights))
                except IndexError: final_layers.add(layer)

            final_layers.compile(optimizer='adam', loss='mse')
            final_layers.build(input_shape=(None, 324))
            # print(final_layers.summary())

            final_models[index] = final_layers
        
    return final_models

In [53]:
indexes = [0]
final_models = std_and_discard(indexes)

In [54]:
predictor_models = [[] if i in indexes else None for i in range(6)]
for i in range(6):
    if predictor_models[i] is not None:
        for j in [3, 5, 7]:
            predictor_models[i].append(Model(inputs=final_models[i].input, outputs=final_models[i].layers[j].output))

# Checking Co-relation

In [55]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  return np.array(new_data)

def predictor(month,mod):
  month_path = slp_months[month]
  month_data = pd.read_csv(month_path)
  feature = get_feature(month_data)
  try: pred_m = models[mod](feature)
  except: 
    print(i)
    raise(IndexError)
  pred_f = pred_m.numpy()
  return pred_f

In [57]:
from sklearn.svm import SVR
from scipy.stats import pearsonr

def get_top_pred(top,pred_info,w_start,w_end):
    pred_f = pred_info
    cor_list = []
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    list1=list(enumerate(cor_list))
    list2=sorted(list1, key=lambda x: x[1],reverse=True)
    top_feature_index = []

    for i in range(top):
        index = list2[i][0]
        top_feature_index.append(index)

    predictor = []
    for i in range(len(pred_f)):
        temp = []
        for j in top_feature_index:
            feature = pred_f[i][j]
            temp.append(feature)
            predictor.append(temp)
    
    return predictor


def get_predictors(months,top,w_start,w_end):
  for i in months:
    for mod in range(len(models)):
      y_pred = predictor(i,mod)
      b = get_top_pred(top, y_pred, w_start,w_end)
      try:    pred = np.concatenate((pred,b), axis=1)
      except: pred = np.array(b)
  return pred
  

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,53-window,53)
    for i in range(14):
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      
      score.append(reg.predict([pred[53+i]])[0])
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)

In [ ]:
# from itertools import combinations
# months_comb = [j for i in range(4) for j in combinations(range(5,9), i+1)]
months_comb = [[4]]
top = 9



minimum, maximum = 1, -1
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print("\n\nMonth combination = ",month,"    top = ",i,"   min and max",left,"   ",right)
    if minimum > left:  minimum, min_month, min_top = left, month, i
    if maximum < right: maximum, max_month, max_top = right, month, i
    print(f'Minimum: {minimum}, {min_month}, {min_top}    Maximum: {maximum}, {max_month}, {max_top}\n')